In [1]:
import torch
import numpy as np
import phi.torch.flow as ptf

import tqdm

device = torch.device('cuda')

In [2]:
BOUNDS = ptf.Box(x=1, y=1, z=1)
RESOLUTION = 100

In [3]:
index1 = 0
index2 = index1 + 5
den1_np = np.load(f'origin/origin_{index1:03d}.npz')['density']
den2_np = np.load(f'origin/origin_{index2:03d}.npz')['density']
den1_gpu = torch.tensor(den1_np, dtype=torch.float32, device=device)
den2_gpu = torch.tensor(den2_np, dtype=torch.float32, device=device)

DENSITY1 = ptf.CenteredGrid(
    values=ptf.wrap(den1_gpu, ptf.spatial('x,y,z')),
    extrapolation=ptf.extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
DENSITY2 = ptf.CenteredGrid(
    values=ptf.wrap(den2_gpu, ptf.spatial('x,y,z')),
    extrapolation=ptf.extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
VELOCITY0 = ptf.StaggeredGrid(
    values=0,
    extrapolation=ptf.extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
INFLOW = ptf.CenteredGrid(
    values=ptf.Sphere(center=ptf.tensor([0.5, 0, 0.5], ptf.channel(vector='x,y,z')), radius=0.05),
    extrapolation=ptf.extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)


# @ptf.jit_compile
def step(v, s, p, dt, inflow_rate, inflow, FORCE):
    s = ptf.advect.mac_cormack(s, v, dt) + inflow_rate * ptf.resample(inflow, to=s, soft=True)
    buoyancy = ptf.resample(s * (0, 0.1, 0), to=v) + ptf.resample(FORCE, to=v)
    v = ptf.advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, p = ptf.fluid.make_incompressible(v, (), ptf.Solve('auto', 1e-3, x0=p))
    return v, s, p


DENSITY = DENSITY1
VELOCITY = VELOCITY0

In [4]:
x = torch.linspace(0.2, 0.8, 3, dtype=torch.float32, device=device)
y = torch.linspace(0.2, 0.8, 3, dtype=torch.float32, device=device)
z = torch.linspace(0.2, 0.8, 3, dtype=torch.float32, device=device)
grid_x, grid_y, grid_z = torch.meshgrid(x, y, z, indexing='ij')

LOCATIONS = torch.stack([grid_x, grid_y, grid_z], dim=-1).reshape(-1, 3)  # [27, 3]
DIRECTIONS = torch.randn_like(LOCATIONS, dtype=torch.float32, device=device, requires_grad=True)  # [27, 3]
OPTIMIZER = torch.optim.RAdam([DIRECTIONS], lr=0.001)

# Gaussian falloff function
A = 0.2  # 高斯幅值
sigma = 0.2  # 标准差，控制衰减范围
LAMBDA_FIELDS = []
for loc in LOCATIONS:
    LAMBDA = ptf.CenteredGrid(
        values=lambda x: A * ptf.math.exp(-ptf.length(x - loc) ** 2 / (2 * sigma ** 2)),
        extrapolation=ptf.extrapolation.ZERO_GRADIENT,
        bounds=BOUNDS,
        resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
    )
    LAMBDA_FIELDS.append(LAMBDA)

DIRECTION_FIELDS = []
for dir in DIRECTIONS:
    DIRECTION = ptf.CenteredGrid(
        values=dir,
        extrapolation=ptf.extrapolation.ZERO_GRADIENT,
        bounds=BOUNDS,
        resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
    )
    DIRECTION_FIELDS.append(DIRECTION)

all_iterations = 100
losses = []
for _ in tqdm.trange(all_iterations):
    OPTIMIZER.zero_grad()

    first = True
    FORCE_FIELD = ptf.CenteredGrid(
        values=(0, 0, 0),
        extrapolation=ptf.extrapolation.ZERO_GRADIENT,
        bounds=BOUNDS,
        resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
    )
    for _1, _2 in zip(LAMBDA_FIELDS, DIRECTION_FIELDS):
        FORCE_FIELD = FORCE_FIELD + _1 * _2

    for frame in range(5):
        VELOCITY, DENSITY, PRESSURE = step(VELOCITY, DENSITY, None, 0.2, 0.2, INFLOW, FORCE_FIELD)


    loss = torch.nn.functional.mse_loss(DENSITY2.values.native('x,y,z'), DENSITY.values.native('x,y,z'))
    loss.backward()
    OPTIMIZER.step()

    print(f'loss: {loss.item()}')
    losses.append(loss.item())

  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\imeho\Documents\VituralEnvs\InstantPINF-TCNN\Lib\site-packages\phiml\backend\torch\_torch_backend.py:803: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  return torch.sparse_csr_tensor(row_pointers, column_indices, values, shape, device=values.device)
  1%|          | 1/100 [00:26<44:25, 26.92s/it]

loss: 0.0003622330550570041


RuntimeError: Output 0 of UnbindBackward0 is a view and its base or another view of its base has been modified inplace. This view is the output of a function that returns multiple views. Such functions do not allow the output views to be modified inplace. You should replace the inplace operation by an out-of-place one.

In [ ]:
x = torch.linspace(0, 1, 3, dtype=torch.float32, device=device)
y = torch.linspace(0, 1, 3, dtype=torch.float32, device=device)
z = torch.linspace(0, 1, 3, dtype=torch.float32, device=device)
grid_x, grid_y, grid_z = torch.meshgrid(x, y, z, indexing='ij')
grid_points = torch.stack([grid_x, grid_y, grid_z], dim=-1)

TO_LEARN_DIRECTION = []
LOCATIONS = []
DIRECTIONS = []
GRAD_vars = []
for pos in grid_points.reshape(-1, 3):
    TO_LEARN_DIRECTION.append(torch.randn(3, device=device, requires_grad=True))
    LOCATIONS.append(ptf.vec(x=pos[0], y=pos[1], z=pos[2]))
    DIRECTIONS.append(ptf.wrap(TO_LEARN_DIRECTION[-1], ptf.channel(vector='x,y,z')))
    GRAD_vars.append(TO_LEARN_DIRECTION[-1])
OPTIMIZER = torch.optim.RAdam(GRAD_vars, lr=0.001)

locations_tensor = torch.tensor([[loc.x, loc.y, loc.z] for loc in LOCATIONS], device=device)  # 形状 [N, 3]
directions_tensor = torch.stack([torch.tensor(dir.native(), device=device) for dir in DIRECTIONS])  # 形状 [N, 3]

losses = []

all_iterations = 100
for _ in tqdm.trange(all_iterations):
    OPTIMIZER.zero_grad()

    FORCE_FIELD = ptf.CenteredGrid(
        values=(0, 0, 0),
        extrapolation=ptf.extrapolation.ZERO_GRADIENT,
        bounds=BOUNDS,
        resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
    )
    for LOCATION, DIRECTION in zip(LOCATIONS, DIRECTIONS):
        LAMBDA = ptf.CenteredGrid(
            values=lambda loc: A * ptf.math.exp(-ptf.length(loc - LOCATION) ** 2 / (2 * sigma ** 2)),
            extrapolation=ptf.extrapolation.ZERO_GRADIENT,
            bounds=BOUNDS,
            resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
        )
        VECTOR_FIELD = ptf.CenteredGrid(
            values=DIRECTION,
            extrapolation=ptf.extrapolation.ZERO_GRADIENT,
            bounds=BOUNDS,
            resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
        )
        FORCE_FIELD = FORCE_FIELD + LAMBDA * VECTOR_FIELD

    for frame in range(5):
        VELOCITY, DENSITY, PRESSURE = step(VELOCITY, DENSITY, None, 0.2, 0.2, INFLOW, FORCE_FIELD)

    loss = torch.nn.functional.mse_loss(DENSITY2.values.native('x,y,z'), DENSITY.values.native('x,y,z'))
    loss.backward(retain_graph=True)
    OPTIMIZER.step()

    print(f'loss: {loss.item()}')
    losses.append(loss.item())

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(range(all_iterations), losses, label='Loss')
plt.xlabel('Step')
plt.ylabel('Loss')
plt.title('Loss Over Steps')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# x = torch.linspace(0, 1, 5, dtype=torch.float32, device=device)
# y = torch.linspace(0, 1, 5, dtype=torch.float32, device=device)
# z = torch.linspace(0, 1, 5, dtype=torch.float32, device=device)
# grid_x, grid_y, grid_z = torch.meshgrid(x, y, z, indexing='ij')
# grid_points = torch.stack([grid_x, grid_y, grid_z], dim=-1)
#
# TO_LEARN_DIRECTION = []
# LOCATIONS = []
# DIRECTIONS = []
# GRAD_vars = []
# for pos in grid_points.reshape(-1, 3):
#     TO_LEARN_DIRECTION.append(torch.randn(3, device=device, requires_grad=True))
#     LOCATIONS.append(ptf.vec(x=pos[0], y=pos[1], z=pos[2]))
#     DIRECTIONS.append(ptf.wrap(TO_LEARN_DIRECTION[-1], ptf.channel(vector='x,y,z')))
#     GRAD_vars.append(TO_LEARN_DIRECTION[-1])
# OPTIMIZER = torch.optim.RAdam(GRAD_vars, lr=0.001)
#
# FORCE_FIELD = ptf.CenteredGrid(
#     values=(0, 0, 0),
#     extrapolation=ptf.extrapolation.ZERO_GRADIENT,
#     bounds=BOUNDS,
#     resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
# )
#
# # Gaussian falloff function
# A = 0.2  # 高斯幅值
# sigma = 0.2  # 标准差，控制衰减范围
# for LOCATION, DIRECTION in tqdm.tqdm(zip(LOCATIONS, DIRECTIONS)):
#     LAMBDA = ptf.CenteredGrid(
#         values=lambda loc: A * ptf.math.exp(-ptf.length(loc - LOCATION) ** 2 / (2 * sigma ** 2)),
#         extrapolation=ptf.extrapolation.ZERO_GRADIENT,
#         bounds=BOUNDS,
#         resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
#     )
#     VECTOR_FIELD = ptf.CenteredGrid(
#         values=DIRECTION,
#         extrapolation=ptf.extrapolation.ZERO_GRADIENT,
#         bounds=BOUNDS,
#         resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
#     )
#     FORCE_FIELD = FORCE_FIELD + LAMBDA * VECTOR_FIELD
